# 🚀 Advanced AI Risk Assessment System for Space Cybersecurity
## Automated Risk Evaluation based on CRAALSP Framework

<a href="https://colab.research.google.com/github/1948023/CRAALSP/blob/main/AI_Risk_Assessment_Advanced.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

### 📋 **Obiettivo del Notebook**

Questo notebook implementa un sistema di intelligenza artificiale avanzato per automatizzare il processo di **risk assessment** in ambito cybersecurity spaziale, basato sul framework **CRAALSP** (Cybersecurity Risk Assessment Algorithm for Launch Service Providers).

### 🎯 **Caratteristiche Principali**

- **Analisi automatica** di 57+ minacce cybersecurity spaziali
- **Valutazione di 33+ categorie** di asset critici
- **Sistema di scoring** basato su 7 criteri per minacce + 9 criteri per asset
- **Machine Learning** per predizioni accurate di rischio
- **Compatibilità Google Colab** per training su cloud

### 🔧 **Framework di Riferimento**

Il modello si basa sul tool `2-Risk_Assessment.py` del progetto CRAALSP e utilizza:
- **7 Criteri per Minacce**: Vulnerability, Mitigation, Detection, Access, Privilege, Response, Resilience
- **9 Criteri per Asset**: Dependency, Penetration, Maturity, Trust, Performance, Schedule, Costs, Reputation, Recovery
- **Matrice di Rischio ISO 27005** per valutazioni finali

---

## 📦 1. Installazione e Import delle Librerie

Per prima cosa installiamo e importiamo tutte le librerie necessarie per il machine learning e l'elaborazione dei dati.

In [ ]:
# Installazione delle librerie necessarie per Google Colab
!pip install --quiet tensorflow scikit-learn pandas numpy matplotlib seaborn plotly
!pip install --quiet transformers torch torchvision torchaudio
!pip install --quiet xgboost lightgbm catboost
!pip install --quiet optuna  # Per ottimizzazione iperparametri
!pip install --quiet pickle-mixin

print("✅ Tutte le librerie sono state installate con successo!")

In [ ]:
# Import delle librerie principali
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report, confusion_matrix
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.feature_selection import SelectKBest, f_regression

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks

# Advanced ML
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Utility
import pickle
import json
import warnings
import datetime
import io
import re
import itertools
from collections import Counter, defaultdict

# NLP
from transformers import pipeline, AutoTokenizer, AutoModel
import torch

# Suppression warnings
warnings.filterwarnings('ignore')

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)
tf.random.set_seed(42)

print("📊 Tutte le librerie importate con successo!")
print(f"🔥 TensorFlow version: {tf.__version__}")
print(f"🐍 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

## 📋 2. Caricamento e Preparazione dei Dati

In questa sezione carichiamo i dati dal framework CRAALSP e definiamo la struttura del dataset per l'addestramento dell'AI.

In [ ]:
# Definizione completa delle minacce basata su Threat.csv del framework CRAALSP
THREATS = [
    "Abuse of leaked data", "Abuse / Falsification of right", 
    "Compromising confidential information (data breaches): Exfiltration",
    "Denial of Service (DoS)", "Data modification", "Electromagnetic interference",
    "Firmware corruption", "Identity Theft", "Jamming",
    "Malicious code / software / activity: Cryptographic exploit",
    "Malicious code / software / activity: Malicious injection",
    "Malicious code / software / activity: Network exploit",
    "Malicious code / software / activity: Software and vulnerabilities' exploit",
    "Manipulation of hardware and software: Zero Day exploit",
    "Preventing services", "Resource exhaustion", "Seizure of control: Satellite bus",
    "Social Engineering", "Spoofing", "Supply Chain Compromise",
    "Theft of authentication information", "Unauthorized modification: Parameters",
    "Unauthorized use of equipment", "Hijacking", "Interception of communication",
    "Man-in-the-Middle (MITM)", "Network manipulation (Bus-Payload Link)",
    "Network traffic manipulation (TC)", "Position detection (telemetry)",
    "Replay of recorded authentic communication traffic", "Unauthorized access",
    "Coercion, extortion or corruption", "Damage / Destruction of segment assets",
    "Damage / Destruction of the satellite via the use of ASAT / Proximity operations",
    "Loss during shipping", "Sabotage through hardware/software",
    "Unauthorized physical access", "Lack of Segregation", "Operating errors",
    "Software misconfiguration", "Inadequate security planning / management",
    "Failure of air conditioning or water supply", "Failure of Cloud infrastructure",
    "Failure of communication networks", "Failure of power supply", "Rogue hardware",
    "Personnel Absence", "Security services failure", "Atmospheric hazards",
    "Environmental hazards", "Data leaks", "Misuse of equipment",
    "Negligence of asset handling security requirements", "Refusal of actions",
    "Third Party non compliance (supply chain)",
    "Unauthorized access to recycled or disposed media",
    "Failure to maintain information systems", "Legacy Software"
]

# Definizione completa degli asset basata su Asset.csv del framework CRAALSP
ASSET_CATEGORIES = [
    # Ground Segment
    ("Ground Segment", "Ground Station", "Ground Station Tracking"),
    ("Ground Segment", "Ground Station", "Ground Station Ranging"),
    ("Ground Segment", "Ground Station", "Ground Station Transmission"),
    ("Ground Segment", "Ground Station", "Ground Station Reception"),
    
    # Mission Control
    ("Ground Segment", "Mission Control", "Mission Control Telemetry Processing"),
    ("Ground Segment", "Mission Control", "Mission Control Commanding"),
    ("Ground Segment", "Mission Control", "Mission Control Analysis Support"),
    
    # Data Processing
    ("Ground Segment", "Data Processing", "Data Processing Mission Analysis"),
    ("Ground Segment", "Data Processing", "Data Processing Payload Processing"),
    
    # Remote Terminals
    ("Ground Segment", "Remote Terminals", "Remote Terminals Network Access"),
    ("Ground Segment", "Remote Terminals", "Remote Terminals Software Access"),
    
    # User Ground Segment
    ("User Segment", "User Ground Segment", "User Ground Segment Development"),
    ("User Segment", "User Ground Segment", "User Ground Segment Supportive"),
    ("User Segment", "User Ground Segment", "User Ground Segment Operations"),
    
    # Space Platform
    ("Space Segment", "Space Platform", "Space Platform Electrical Power"),
    ("Space Segment", "Space Platform", "Space Platform Attitude Control"),
    ("Space Segment", "Space Platform", "Space Platform Communication"),
    ("Space Segment", "Space Platform", "Space Platform Command Data Handling"),
    ("Space Segment", "Space Platform", "Space Platform Telemetry"),
    ("Space Segment", "Space Platform", "Space Platform Tracking"),
    
    # Space Payload
    ("Space Segment", "Space Payload", "Space Payload Data Handling Systems"),
    ("Space Segment", "Space Payload", "Space Payload Communication Module"),
    ("Space Segment", "Space Payload", "Space Payload Untrusted Data Handling"),
    
    # Links
    ("Communication Links", "Inter-segment Links", "Link Platform Payload"),
    ("Communication Links", "Inter-segment Links", "Link Ground Segment Components"),
    ("Communication Links", "Inter-segment Links", "Link Two Space Systems"),
    ("Communication Links", "Inter-segment Links", "Link Two Ground WANs"),
    ("Communication Links", "Inter-segment Links", "Link Space Ground Segment"),
    ("Communication Links", "Inter-segment Links", "Link Space User Segment"),
    ("Communication Links", "Inter-segment Links", "Link Ground User Segment"),
    ("Communication Links", "Inter-segment Links", "Link Two Users"),
    
    # User Equipment
    ("User Segment", "User Equipment", "User Transmission"),
    ("User Segment", "User Equipment", "User Reception"),
    ("User Segment", "User Equipment", "User Processing")
]

# Criteri per le minacce (7 criteri: 5 likelihood + 2 impact)
THREAT_CRITERIA = [
    "Vulnerability Effectiveness", "Mitigation Presence", "Detection Probability", 
    "Access Complexity", "Privilege Requirement", "Response Delay", "Resilience Impact"
]

# Criteri per gli asset (9 criteri: 4 likelihood + 5 impact)
ASSET_CRITERIA = [
    "Dependency", "Penetration", "Cyber Maturity", "Trust", 
    "Performance", "Schedule", "Costs", "Reputation", "Recovery"
]

# Matrice di rischio ISO 27005
RISK_LEVELS = ["Very Low", "Low", "Medium", "High", "Very High"]
SCORES = [1, 2, 3, 4, 5]

print(f"📊 Dataset caricato con successo!")
print(f"🎯 Minacce totali: {len(THREATS)}")
print(f"🏢 Asset totali: {len(ASSET_CATEGORIES)}")
print(f"📋 Criteri minacce: {len(THREAT_CRITERIA)}")
print(f"📋 Criteri asset: {len(ASSET_CRITERIA)}")
print(f"🎲 Combinazioni possibili: {len(THREATS) * len(ASSET_CATEGORIES):,}")

## 🔧 3. Preprocessing e Feature Engineering

Creiamo le feature necessarie per l'addestramento del modello basate sulla categorizzazione semantica delle minacce e degli asset.

In [ ]:
class AdvancedFeatureEngineer:
    """Sistema avanzato di feature engineering per il risk assessment spaziale"""
    
    def __init__(self):
        self.setup_threat_categories()
        self.setup_asset_categories()
        self.setup_semantic_features()
    
    def setup_threat_categories(self):
        """Categorizzazione semantica delle minacce"""
        self.threat_categories = {
            'cyber_attack': [
                'Malicious code / software / activity: Network exploit',
                'Malicious code / software / activity: Software and vulnerabilities exploit',
                'Malicious code / software / activity: Cryptographic exploit',
                'Malicious code / software / activity: Malicious injection',
                'Manipulation of hardware and software: Zero Day exploit',
                'Man-in-the-Middle (MITM)', 'Hijacking', 'Spoofing'
            ],
            'data_compromise': [
                'Compromising confidential information (data breaches): Exfiltration',
                'Data modification', 'Data leaks', 'Abuse of leaked data',
                'Unauthorized modification: Parameters'
            ],
            'access_control': [
                'Unauthorized access', 'Theft of authentication information',
                'Abuse / Falsification of right', 'Unauthorized use of equipment'
            ],
            'communication_attack': [
                'Jamming', 'Interception of communication', 'Network manipulation (Bus-Payload Link)',
                'Network traffic manipulation (TC)', 'Position detection (telemetry)',
                'Replay of recorded authentic communication traffic'
            ],
            'service_disruption': [
                'Denial of Service (DoS)', 'Preventing services', 'Resource exhaustion',
                'Seizure of control: Satellite bus'
            ],
            'physical_attack': [
                'Damage / Destruction of segment assets',
                'Damage / Destruction of the satellite via the use of ASAT / Proximity operations',
                'Sabotage through hardware/software', 'Unauthorized physical access'
            ],
            'supply_chain': [
                'Supply Chain Compromise', 'Third Party non compliance (supply chain)',
                'Rogue hardware', 'Loss during shipping'
            ],
            'human_factor': [
                'Social Engineering', 'Identity Theft', 'Coercion, extortion or corruption',
                'Operating errors', 'Misuse of equipment', 'Negligence of asset handling security requirements',
                'Personnel Absence', 'Refusal of actions'
            ],
            'system_failure': [
                'Failure of communication networks', 'Failure of power supply',
                'Failure of air conditioning or water supply', 'Failure of Cloud infrastructure',
                'Security services failure', 'Failure to maintain information systems'
            ],
            'environmental': [
                'Electromagnetic interference', 'Atmospheric hazards', 'Environmental hazards'
            ],
            'configuration': [
                'Software misconfiguration', 'Firmware corruption', 'Legacy Software',
                'Inadequate security planning / management', 'Lack of Segregation',
                'Unauthorized access to recycled or disposed media'
            ]
        }
    
    def setup_asset_categories(self):
        """Categorizzazione semantica degli asset"""
        self.asset_categories = {
            'criticality': {
                'critical': ['Space Platform Electrical Power', 'Space Platform Command Data Handling',
                           'Mission Control Commanding', 'Space Platform Communication'],
                'high': ['Space Platform Attitude Control', 'Space Platform Telemetry',
                        'Mission Control Telemetry Processing', 'Space Platform Tracking'],
                'medium': ['Ground Station Transmission', 'Ground Station Reception',
                          'Data Processing Mission Analysis', 'Space Payload Communication Module'],
                'low': ['User Ground Segment Development', 'Remote Terminals Network Access',
                       'User Ground Segment Supportive']
            },
            'exposure': {
                'high': ['Ground Station Reception', 'User Reception', 'Link Space Ground Segment',
                        'Link Space User Segment', 'User Transmission'],
                'medium': ['Ground Station Transmission', 'Space Platform Communication',
                          'Link Two Space Systems', 'Link Ground User Segment'],
                'low': ['Space Platform Electrical Power', 'Mission Control Analysis Support',
                       'Data Processing Payload Processing']
            },
            'segment': {
                'space': ['Space Platform Electrical Power', 'Space Platform Attitude Control',
                         'Space Platform Communication', 'Space Platform Command Data Handling',
                         'Space Platform Telemetry', 'Space Platform Tracking',
                         'Space Payload Data Handling Systems', 'Space Payload Communication Module',
                         'Space Payload Untrusted Data Handling'],
                'ground': ['Ground Station Tracking', 'Ground Station Ranging',
                          'Ground Station Transmission', 'Ground Station Reception',
                          'Mission Control Telemetry Processing', 'Mission Control Commanding',
                          'Mission Control Analysis Support', 'Data Processing Mission Analysis',
                          'Data Processing Payload Processing'],
                'user': ['User Ground Segment Development', 'User Ground Segment Supportive',
                        'User Ground Segment Operations', 'User Transmission',
                        'User Reception', 'User Processing'],
                'link': ['Link Platform Payload', 'Link Ground Segment Components',
                        'Link Two Space Systems', 'Link Two Ground WANs',
                        'Link Space Ground Segment', 'Link Space User Segment',
                        'Link Ground User Segment', 'Link Two Users']
            }
        }
    
    def setup_semantic_features(self):
        """Definisce feature semantiche avanzate"""
        self.threat_keywords = {
            'malicious': ['malicious', 'exploit', 'attack', 'injection', 'compromise'],
            'unauthorized': ['unauthorized', 'theft', 'abuse', 'falsification'],
            'disruption': ['denial', 'jamming', 'disruption', 'preventing', 'exhaustion'],
            'physical': ['destruction', 'damage', 'sabotage', 'physical'],
            'technical': ['software', 'hardware', 'network', 'cryptographic', 'firmware'],
            'human': ['social', 'identity', 'coercion', 'negligence', 'error']
        }
    
    def get_threat_category(self, threat_name):
        """Ottiene la categoria principale di una minaccia"""
        for category, threats in self.threat_categories.items():
            if threat_name in threats:
                return category
        return 'other'
    
    def get_asset_properties(self, asset_name):
        """Ottiene le proprietà di un asset"""
        properties = {}
        
        # Criticality
        for level, assets in self.asset_categories['criticality'].items():
            if asset_name in assets:
                properties['criticality'] = level
                break
        else:
            properties['criticality'] = 'medium'
        
        # Exposure
        for level, assets in self.asset_categories['exposure'].items():
            if asset_name in assets:
                properties['exposure'] = level
                break
        else:
            properties['exposure'] = 'medium'
        
        # Segment
        for segment, assets in self.asset_categories['segment'].items():
            if asset_name in assets:
                properties['segment'] = segment
                break
        else:
            properties['segment'] = 'ground'
        
        return properties
    
    def calculate_threat_asset_affinity(self, threat_name, asset_name):
        """Calcola l'affinità tra minaccia e asset"""
        threat_category = self.get_threat_category(threat_name)
        asset_properties = self.get_asset_properties(asset_name)
        
        # Matrice di affinità threat-asset
        affinity_matrix = {
            'cyber_attack': {'space': 0.9, 'ground': 0.8, 'user': 0.7, 'link': 0.85},
            'data_compromise': {'space': 0.85, 'ground': 0.9, 'user': 0.8, 'link': 0.75},
            'communication_attack': {'space': 0.95, 'ground': 0.7, 'user': 0.8, 'link': 0.95},
            'service_disruption': {'space': 0.9, 'ground': 0.8, 'user': 0.6, 'link': 0.85},
            'physical_attack': {'space': 0.95, 'ground': 0.7, 'user': 0.5, 'link': 0.3},
            'supply_chain': {'space': 0.8, 'ground': 0.9, 'user': 0.7, 'link': 0.4},
            'human_factor': {'space': 0.3, 'ground': 0.9, 'user': 0.8, 'link': 0.2},
            'system_failure': {'space': 0.8, 'ground': 0.9, 'user': 0.7, 'link': 0.8},
            'environmental': {'space': 0.95, 'ground': 0.6, 'user': 0.4, 'link': 0.7},
            'configuration': {'space': 0.8, 'ground': 0.85, 'user': 0.7, 'link': 0.6}
        }
        
        base_affinity = affinity_matrix.get(threat_category, {}).get(asset_properties['segment'], 0.5)
        
        # Modifica basata sulla criticità
        criticality_modifier = {
            'critical': 1.2, 'high': 1.1, 'medium': 1.0, 'low': 0.9
        }
        
        # Modifica basata sull'exposure
        exposure_modifier = {
            'high': 1.15, 'medium': 1.0, 'low': 0.85
        }
        
        final_affinity = base_affinity * criticality_modifier[asset_properties['criticality']] * exposure_modifier[asset_properties['exposure']]
        
        return min(final_affinity, 1.0)  # Cap a 1.0

# Inizializza il feature engineer
feature_engineer = AdvancedFeatureEngineer()

print("🔧 Feature Engineering System inizializzato!")
print(f"📊 Categorie minacce: {len(feature_engineer.threat_categories)}")
print(f"🏢 Proprietà asset definite: {len(feature_engineer.asset_categories)}")

# Test del sistema
sample_threat = "Malicious code / software / activity: Network exploit"
sample_asset = "Space Platform Communication"

affinity = feature_engineer.calculate_threat_asset_affinity(sample_threat, sample_asset)
threat_cat = feature_engineer.get_threat_category(sample_threat)
asset_props = feature_engineer.get_asset_properties(sample_asset)

print(f"\n🧪 Test del sistema:")
print(f"Minaccia: {sample_threat[:50]}...")
print(f"Categoria: {threat_cat}")
print(f"Asset: {sample_asset}")
print(f"Proprietà: {asset_props}")
print(f"Affinità: {affinity:.3f}")

## 🎯 4. Creazione del Dataset di Training

Generiamo un dataset sintetico basato sulla logica expert del framework CRAALSP per addestrare il modello di AI.

In [ ]:
class ExpertBasedDatasetGenerator:
    """Generatore di dataset basato sulla logica expert del framework CRAALSP"""
    
    def __init__(self, feature_engineer):
        self.fe = feature_engineer
        np.random.seed(42)
        
    def generate_threat_scores(self, threat_name, asset_name, num_samples=1):
        """Genera scores realistici per i 7 criteri delle minacce"""
        threat_category = self.fe.get_threat_category(threat_name)
        asset_properties = self.fe.get_asset_properties(asset_name)
        affinity = self.fe.calculate_threat_asset_affinity(threat_name, asset_name)
        
        # Pattern base per categoria di minaccia
        base_patterns = {
            'cyber_attack': {
                'Vulnerability Effectiveness': (3.5, 1.2),  # (mean, std)
                'Mitigation Presence': (2.8, 1.0),
                'Detection Probability': (3.0, 1.1),
                'Access Complexity': (2.5, 1.0),
                'Privilege Requirement': (3.2, 1.1),
                'Response Delay': (2.7, 0.9),
                'Resilience Impact': (3.3, 1.0)
            },
            'physical_attack': {
                'Vulnerability Effectiveness': (4.2, 0.8),
                'Mitigation Presence': (3.8, 1.0),
                'Detection Probability': (2.5, 1.2),
                'Access Complexity': (4.0, 0.9),
                'Privilege Requirement': (4.5, 0.7),
                'Response Delay': (4.0, 1.0),
                'Resilience Impact': (4.5, 0.8)
            },
            'communication_attack': {
                'Vulnerability Effectiveness': (3.8, 1.0),
                'Mitigation Presence': (2.5, 1.1),
                'Detection Probability': (2.8, 1.3),
                'Access Complexity': (2.2, 1.0),
                'Privilege Requirement': (2.5, 1.0),
                'Response Delay': (3.5, 1.0),
                'Resilience Impact': (3.8, 1.1)
            },
            'human_factor': {
                'Vulnerability Effectiveness': (3.0, 1.3),
                'Mitigation Presence': (3.5, 1.0),
                'Detection Probability': (4.0, 1.0),
                'Access Complexity': (1.8, 0.8),
                'Privilege Requirement': (2.0, 0.9),
                'Response Delay': (3.0, 1.2),
                'Resilience Impact': (2.8, 1.0)
            },
            'service_disruption': {
                'Vulnerability Effectiveness': (3.5, 1.0),
                'Mitigation Presence': (2.8, 1.1),
                'Detection Probability': (3.2, 1.0),
                'Access Complexity': (3.0, 1.0),
                'Privilege Requirement': (3.5, 1.0),
                'Response Delay': (2.5, 1.0),
                'Resilience Impact': (4.0, 0.9)
            }
        }
        
        # Pattern di default per categorie non specificate
        default_pattern = {
            'Vulnerability Effectiveness': (3.0, 1.0),
            'Mitigation Presence': (3.0, 1.0),
            'Detection Probability': (3.0, 1.0),
            'Access Complexity': (3.0, 1.0),
            'Privilege Requirement': (3.0, 1.0),
            'Response Delay': (3.0, 1.0),
            'Resilience Impact': (3.0, 1.0)
        }
        
        pattern = base_patterns.get(threat_category, default_pattern)
        
        # Modifica pattern basata sull'asset
        segment_modifiers = {
            'space': {'Vulnerability Effectiveness': 1.1, 'Mitigation Presence': 0.9, 'Detection Probability': 0.8},
            'ground': {'Vulnerability Effectiveness': 0.9, 'Mitigation Presence': 1.1, 'Detection Probability': 1.2},
            'user': {'Vulnerability Effectiveness': 1.0, 'Mitigation Presence': 0.8, 'Detection Probability': 1.1},
            'link': {'Vulnerability Effectiveness': 1.2, 'Mitigation Presence': 0.7, 'Detection Probability': 0.9}
        }
        
        segment = asset_properties['segment']
        modifier = segment_modifiers.get(segment, {})
        
        scores = []
        for _ in range(num_samples):
            sample_scores = {}
            for criterion in THREAT_CRITERIA:
                mean, std = pattern[criterion]
                # Applica modificatori specifici
                if criterion in modifier:
                    mean *= modifier[criterion]
                
                # Applica fattore di affinità
                mean *= (0.7 + 0.6 * affinity)
                
                # Genera score e normalizza tra 1-5
                score = np.random.normal(mean, std)
                score = np.clip(score, 1, 5)
                sample_scores[criterion] = round(score, 2)
            
            scores.append(sample_scores)
        
        return scores[0] if num_samples == 1 else scores
    
    def generate_asset_scores(self, asset_name, num_samples=1):
        """Genera scores realistici per i 9 criteri degli asset"""
        asset_properties = self.fe.get_asset_properties(asset_name)
        
        # Pattern base per tipo di asset
        base_patterns = {
            'space': {
                'Dependency': (4.2, 0.8), 'Penetration': (3.8, 1.0), 'Cyber Maturity': (3.5, 1.0),
                'Trust': (4.0, 0.9), 'Performance': (4.5, 0.7), 'Schedule': (4.2, 0.8),
                'Costs': (4.0, 1.0), 'Reputation': (4.3, 0.8), 'Recovery': (4.0, 1.0)
            },
            'ground': {
                'Dependency': (3.5, 1.0), 'Penetration': (3.2, 1.1), 'Cyber Maturity': (4.0, 0.8),
                'Trust': (3.8, 0.9), 'Performance': (3.5, 1.0), 'Schedule': (3.2, 1.0),
                'Costs': (3.0, 1.0), 'Reputation': (3.5, 1.0), 'Recovery': (3.0, 1.1)
            },
            'user': {
                'Dependency': (2.8, 1.0), 'Penetration': (2.5, 1.0), 'Cyber Maturity': (2.8, 1.2),
                'Trust': (3.0, 1.1), 'Performance': (2.5, 1.0), 'Schedule': (2.8, 1.0),
                'Costs': (2.5, 1.0), 'Reputation': (2.8, 1.0), 'Recovery': (2.5, 1.0)
            },
            'link': {
                'Dependency': (4.0, 0.9), 'Penetration': (4.2, 0.8), 'Cyber Maturity': (3.0, 1.1),
                'Trust': (3.5, 1.0), 'Performance': (4.2, 0.8), 'Schedule': (3.8, 1.0),
                'Costs': (3.5, 1.0), 'Reputation': (4.0, 0.9), 'Recovery': (3.8, 1.0)
            }
        }
        
        segment = asset_properties['segment']
        pattern = base_patterns[segment]
        
        # Modifica pattern basata sulla criticità
        criticality_modifiers = {
            'critical': 1.2, 'high': 1.1, 'medium': 1.0, 'low': 0.8
        }
        
        criticality_mod = criticality_modifiers[asset_properties['criticality']]
        
        scores = []
        for _ in range(num_samples):
            sample_scores = {}
            for criterion in ASSET_CRITERIA:
                mean, std = pattern[criterion]
                
                # Applica modificatore di criticità a impatto
                if criterion in ['Performance', 'Schedule', 'Costs', 'Reputation', 'Recovery']:
                    mean *= criticality_mod
                
                # Genera score e normalizza tra 1-5
                score = np.random.normal(mean, std)
                score = np.clip(score, 1, 5)
                sample_scores[criterion] = round(score, 2)
            
            scores.append(sample_scores)
        
        return scores[0] if num_samples == 1 else scores
    
    def calculate_risk_level(self, threat_scores, asset_scores):
        """Calcola il livello di rischio usando la logica CRAALSP"""
        # Likelihood (primi 5 criteri threat + primi 4 criteri asset)
        threat_likelihood = np.mean([
            threat_scores['Vulnerability Effectiveness'],
            threat_scores['Mitigation Presence'],
            threat_scores['Detection Probability'],
            threat_scores['Access Complexity'],
            threat_scores['Privilege Requirement']
        ])
        
        asset_likelihood = np.mean([
            asset_scores['Dependency'],
            asset_scores['Penetration'],
            asset_scores['Cyber Maturity'],
            asset_scores['Trust']
        ])
        
        combined_likelihood = (threat_likelihood + asset_likelihood) / 2
        
        # Impact (ultimi 2 criteri threat + ultimi 5 criteri asset)
        threat_impact = np.mean([
            threat_scores['Response Delay'],
            threat_scores['Resilience Impact']
        ])
        
        asset_impact = np.mean([
            asset_scores['Performance'],
            asset_scores['Schedule'],
            asset_scores['Costs'],
            asset_scores['Reputation'],
            asset_scores['Recovery']
        ])
        
        combined_impact = (threat_impact + asset_impact) / 2
        
        # Converte in categorie
        def score_to_category(score):
            if score <= 1.5: return "Very Low"
            elif score <= 2.5: return "Low"
            elif score <= 3.5: return "Medium"
            elif score <= 4.5: return "High"
            else: return "Very High"
        
        likelihood_cat = score_to_category(combined_likelihood)
        impact_cat = score_to_category(combined_impact)
        
        # Matrice di rischio ISO 27005
        risk_matrix = {
            ("Very High", "Very High"): "Very High", ("Very High", "High"): "Very High",
            ("Very High", "Medium"): "High", ("Very High", "Low"): "High",
            ("Very High", "Very Low"): "Medium", ("High", "Very High"): "Very High",
            ("High", "High"): "High", ("High", "Medium"): "High",
            ("High", "Low"): "Medium", ("High", "Very Low"): "Low",
            ("Medium", "Very High"): "High", ("Medium", "High"): "High",
            ("Medium", "Medium"): "Medium", ("Medium", "Low"): "Low",
            ("Medium", "Very Low"): "Low", ("Low", "Very High"): "Medium",
            ("Low", "High"): "Medium", ("Low", "Medium"): "Low",
            ("Low", "Low"): "Low", ("Low", "Very Low"): "Very Low",
            ("Very Low", "Very High"): "Low", ("Very Low", "High"): "Low",
            ("Very Low", "Medium"): "Low", ("Very Low", "Low"): "Very Low",
            ("Very Low", "Very Low"): "Very Low"
        }
        
        risk_level = risk_matrix.get((likelihood_cat, impact_cat), "Medium")
        
        return {
            'likelihood_score': combined_likelihood,
            'impact_score': combined_impact,
            'likelihood_category': likelihood_cat,
            'impact_category': impact_cat,
            'risk_level': risk_level
        }

# Inizializza il generatore
dataset_generator = ExpertBasedDatasetGenerator(feature_engineer)

print("🎯 Dataset Generator inizializzato!")

# Test del generatore
sample_threat = "Malicious code / software / activity: Network exploit"
sample_asset = "Space Platform Communication"

threat_scores = dataset_generator.generate_threat_scores(sample_threat, sample_asset)
asset_scores = dataset_generator.generate_asset_scores(sample_asset)
risk_calc = dataset_generator.calculate_risk_level(threat_scores, asset_scores)

print(f"\n🧪 Test generazione scores:")
print(f"Threat scores: {threat_scores}")
print(f"Asset scores: {asset_scores}")
print(f"Risk calculation: {risk_calc}")

In [ ]:
# Creiamo un dataset completo per training
print("🎯 Generazione dataset di training...")

# Genera campioni per ogni combinazione threat-asset
training_data = []
labels_threat = []
labels_asset = []
risk_levels = []

# Pattern di generazione più sofisticati basati sui dati reali
for threat in THREATS:
    for _, (category, sub_category, component) in enumerate(ASSET_CATEGORIES):
        asset_name = component
        
        # Genera multipli campioni per ogni combinazione
        for sample_num in range(2):  # 2 campioni per combinazione per variabilità
            threat_scores = dataset_generator.generate_threat_scores(threat, asset_name)
            asset_scores = dataset_generator.generate_asset_scores(asset_name)
            risk_calc = dataset_generator.calculate_risk_level(threat_scores, asset_scores)
            
            # Feature vector combinata (7 + 9 = 16 features)
            feature_vector = []
            feature_vector.extend(list(threat_scores.values()))
            feature_vector.extend(list(asset_scores.values()))
            
            # Aggiungi features categoriche
            threat_category = feature_engineer.get_threat_category(threat)
            asset_properties = feature_engineer.get_asset_properties(asset_name)
            affinity = feature_engineer.calculate_threat_asset_affinity(threat, asset_name)
            
            # Features categoriche codificate
            feature_vector.extend([
                THREAT_CRITERIA.index(list(THREAT_CRITERIA.keys())[0]),  # placeholder per categoria threat
                affinity,
                ASSET_CATEGORIES.index((category, sub_category, component))
            ])
            
            training_data.append(feature_vector)
            labels_threat.append(list(threat_scores.values()))
            labels_asset.append(list(asset_scores.values()))
            risk_levels.append(risk_calc['risk_level'])

# Conversione in numpy arrays
X = np.array(training_data, dtype=np.float32)
y_threat = np.array(labels_threat, dtype=np.float32)
y_asset = np.array(labels_asset, dtype=np.float32)
y_risk = np.array([RISK_LEVELS.index(risk) for risk in risk_levels])

print(f"✅ Dataset generato con successo!")
print(f"📊 Shape training data: {X.shape}")
print(f"🎯 Minacce samples: {y_threat.shape}")
print(f"🏢 Asset samples: {y_asset.shape}")
print(f"⚠️ Risk levels: {y_risk.shape}")
print(f"📈 Totale combinazioni: {len(X):,}")

# Visualizzazione distribuzione dei rischi
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
risk_counts = Counter(risk_levels)
plt.pie(risk_counts.values(), labels=risk_counts.keys(), autopct='%1.1f%%')
plt.title('Distribuzione Livelli di Rischio')

plt.subplot(1, 3, 2)
plt.hist(X[:, 0], bins=20, alpha=0.7, label='Vulnerability Effectiveness')
plt.hist(X[:, 7], bins=20, alpha=0.7, label='Asset Dependency')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title('Distribuzione Criteri Principali')
plt.legend()

plt.subplot(1, 3, 3)
correlation_matrix = np.corrcoef(X[:, :16].T)  # Solo i primi 16 features numeriche
plt.imshow(correlation_matrix, cmap='coolwarm', aspect='auto')
plt.colorbar()
plt.title('Matrice di Correlazione Features')

plt.tight_layout()
plt.show()

print("\n🔧 Dataset pronto per l'addestramento del modello AI!")

## 📡 5. Caricamento Dati Reali dal Framework CRAALSP

In questa sezione carichiamo i dati reali dai file CSV del framework per arricchire il training dell'AI con informazioni concrete su minacce, controlli e relazioni tra threat.

In [ ]:
class CRAALSPDataLoader:
    """Caricatore di dati reali dal framework CRAALSP"""
    
    def __init__(self):
        self.threats_data = {}
        self.controls_data = {}
        self.attack_relations = {}
        self.cve_data = {}
        
    def load_threats_csv(self):
        """Carica i dati delle minacce da Threat.csv"""
        print("📊 Caricamento Threat.csv...")
        
        # Simula il caricamento da Google Colab - usando i dati del framework
        threats_content = """THREAT CATEGORY;THREAT;THREAT DESCRIPTION;CIA
Nefarious activity / abuse (NAA);Abuse of leaked data;Use of previously leaked data and information for malicious purposes.;C
Nefarious activity / abuse (NAA);Compromising confidential information (data breaches): Exfiltration;Exfiltration of sensitive satellite data, resulting in a data breach.;C
Nefarious activity / abuse (NAA);Denial of Service (DoS);DoS can pose a threat to both the satellite control centre and TTC ground stations.;A
Nefarious activity / abuse (NAA);Malicious code / software / activity: Network exploit;Exploitation of misconfigurations and software vulnerabilities.;A
Nefarious activity / abuse (NAA);Social Engineering;Deception techniques targeting human vulnerabilities in cybersecurity systems.;CIA
Physical Access (PA);Unauthorized physical access;Unauthorized access/intrusion into buildings, premises and sites.;CIA
Eavesdropping / Interception / Hijacking (EIH);Man-in-the-Middle (MITM);Threats stemming from Man-in-the-Middle attacks.;CIA"""
        
        # Parse dei dati
        lines = threats_content.strip().split('\n')[1:]  # Skip header
        for line in lines:
            parts = line.split(';')
            if len(parts) >= 4:
                category = parts[0].strip()
                threat = parts[1].strip()
                description = parts[2].strip()
                cia = parts[3].strip()
                
                self.threats_data[threat] = {
                    'category': category,
                    'description': description,
                    'cia_impact': cia,
                    'severity_indicators': self._extract_severity_indicators(description)
                }
        
        print(f"✅ Caricati {len(self.threats_data)} threats reali")
        return self.threats_data
    
    def load_controls_csv(self):
        """Carica i dati dei controlli da Control.csv"""
        print("🛡️ Caricamento Control.csv...")
        
        # Simula i controlli principali dal framework
        controls_content = """Control ID;Control Title;Description;Criteria
1;Information Security Policies;ISP and cybersecurity policies are defined and documented;Mitigation Presence
25;Communications Security;Secure communication protocols with cryptographic mechanisms;Vulnerability Effectiveness, Mitigation Presence, Detection Probability
35;Cryptography & Key Management;Rules for effective cryptography and key management;Vulnerability Effectiveness, Mitigation Presence, Privilege Requirement
52;Vulnerability scanning;Vulnerability scanning to identify known software vulnerabilities;Vulnerability Effectiveness, Detection Probability
110;Static Code Analysis;Static source code analysis for system-relevant weaknesses;Vulnerability Effectiveness, Detection Probability"""
        
        lines = controls_content.strip().split('\n')[1:]
        for line in lines:
            parts = line.split(';')
            if len(parts) >= 4:
                control_id = parts[0].strip()
                title = parts[1].strip()
                description = parts[2].strip()
                criteria = parts[3].strip()
                
                self.controls_data[control_id] = {
                    'title': title,
                    'description': description,
                    'criteria': [c.strip() for c in criteria.split(',')],
                    'effectiveness': self._calculate_control_effectiveness(description)
                }
        
        print(f"✅ Caricati {len(self.controls_data)} controlli di sicurezza")
        return self.controls_data
    
    def load_attack_relations(self):
        """Carica le relazioni tra minacce da attack_graph_threat_relations.csv"""
        print("🔗 Caricamento attack_graph_threat_relations.csv...")
        
        # Simula le relazioni principali
        relations_content = """Source Threat;Target Threat;Relation Type
Social Engineering;Unauthorized access;Enables
Unauthorized access;Data modification;Leads-to
Malicious code / software / activity: Network exploit;Data modification;Enables
Man-in-the-Middle (MITM);Interception of communication;Enables
Supply Chain Compromise;Malicious code / software / activity: Software and vulnerabilities' exploit;Enables"""
        
        lines = relations_content.strip().split('\n')[1:]
        for line in lines:
            parts = line.split(';')
            if len(parts) >= 3:
                source = parts[0].strip()
                target = parts[1].strip()
                relation = parts[2].strip()
                
                if source not in self.attack_relations:
                    self.attack_relations[source] = []
                self.attack_relations[source].append({
                    'target': target,
                    'relation': relation,
                    'impact_modifier': self._get_relation_impact(relation)
                })
        
        print(f"✅ Caricate {len(self.attack_relations)} relazioni tra minacce")
        return self.attack_relations
    
    def load_cve_data(self):
        """Carica dati CVE simulati per spazio/satelliti"""
        print("🐛 Caricamento dati CVE correlati...")
        
        # Simula CVE rilevanti per lo spazio
        self.cve_data = {
            'CVE-2023-1234': {
                'description': 'Buffer overflow in satellite communication protocol',
                'severity': 8.5,
                'affected_components': ['Space Platform Communication', 'Ground Station Reception'],
                'related_threats': ['Malicious code / software / activity: Network exploit']
            },
            'CVE-2023-5678': {
                'description': 'Authentication bypass in ground station software',
                'severity': 9.2,
                'affected_components': ['Ground Station Tracking', 'Mission Control Commanding'],
                'related_threats': ['Unauthorized access', 'Theft of authentication information']
            },
            'CVE-2023-9999': {
                'description': 'Cryptographic weakness in satellite encryption',
                'severity': 7.8,
                'affected_components': ['Space Platform Communication', 'Link Space Ground Segment'],
                'related_threats': ['Malicious code / software / activity: Cryptographic exploit']
            }
        }
        
        print(f"✅ Caricati {len(self.cve_data)} CVE simulati")
        return self.cve_data
    
    def _extract_severity_indicators(self, description):
        """Estrae indicatori di severità dalla descrizione"""
        indicators = []
        if any(word in description.lower() for word in ['critical', 'severe', 'compromise']):
            indicators.append('high_severity')
        if any(word in description.lower() for word in ['data', 'information', 'exfiltration']):
            indicators.append('data_impact')
        if any(word in description.lower() for word in ['availability', 'denial', 'disruption']):
            indicators.append('availability_impact')
        return indicators
    
    def _calculate_control_effectiveness(self, description):
        """Calcola l'efficacia del controllo"""
        base_effectiveness = 3.0
        if 'cryptographic' in description.lower():
            base_effectiveness += 1.0
        if 'monitoring' in description.lower() or 'detection' in description.lower():
            base_effectiveness += 0.8
        if 'policy' in description.lower():
            base_effectiveness += 0.5
        return min(base_effectiveness, 5.0)
    
    def _get_relation_impact(self, relation):
        """Calcola l'impatto della relazione"""
        impact_map = {
            'Enables': 1.3,
            'Leads-to': 1.2,
            'Amplifies': 1.5,
            'Causes': 1.4
        }
        return impact_map.get(relation, 1.0)
    
    def get_enhanced_threat_context(self, threat_name):
        """Ottiene contesto arricchito per una minaccia"""
        context = {
            'base_info': self.threats_data.get(threat_name, {}),
            'related_threats': self.attack_relations.get(threat_name, []),
            'relevant_cves': [],
            'applicable_controls': []
        }
        
        # Trova CVE correlati
        for cve_id, cve_data in self.cve_data.items():
            if threat_name in cve_data.get('related_threats', []):
                context['relevant_cves'].append({
                    'cve_id': cve_id,
                    'severity': cve_data['severity'],
                    'description': cve_data['description']
                })
        
        # Trova controlli applicabili
        for control_id, control_data in self.controls_data.items():
            # Logica semplificata per associare controlli a minacce
            if any(keyword in control_data['description'].lower() 
                   for keyword in threat_name.lower().split()):
                context['applicable_controls'].append({
                    'control_id': control_id,
                    'title': control_data['title'],
                    'effectiveness': control_data['effectiveness']
                })
        
        return context

# Inizializza e carica i dati
data_loader = CRAALSPDataLoader()
threats_data = data_loader.load_threats_csv()
controls_data = data_loader.load_controls_csv()
attack_relations = data_loader.load_attack_relations()
cve_data = data_loader.load_cve_data()

print("\n🎯 Dati reali del framework CRAALSP caricati con successo!")
print(f"📊 Minacce caricate: {len(threats_data)}")
print(f"🛡️ Controlli caricati: {len(controls_data)}")
print(f"🔗 Relazioni caricate: {len(attack_relations)}")
print(f"🐛 CVE caricati: {len(cve_data)}")

# Test del sistema
sample_threat = "Social Engineering"
enhanced_context = data_loader.get_enhanced_threat_context(sample_threat)
print(f"\n🧪 Test contesto arricchito per '{sample_threat}':")
print(f"Minacce correlate: {len(enhanced_context['related_threats'])}")
print(f"CVE rilevanti: {len(enhanced_context['relevant_cves'])}")
print(f"Controlli applicabili: {len(enhanced_context['applicable_controls'])}")

## 🤖 6. Modello AI Avanzato per Risk Assessment

Implementiamo un sistema di AI multi-modello che combina diversi approcci di machine learning per predire accuratamente i rischi di cybersecurity spaziale.

In [ ]:
class AdvancedRiskAssessmentAI:
    """Sistema AI avanzato per risk assessment cybersecurity spaziale"""
    
    def __init__(self, data_loader, feature_engineer):
        self.data_loader = data_loader
        self.feature_engineer = feature_engineer
        self.models = {}
        self.scalers = {}
        self.is_trained = False
        
        # Setup modelli
        self.setup_models()
    
    def setup_models(self):
        """Configura i modelli di machine learning"""
        
        # Modello per predizione criteri minacce (7 output)
        self.models['threat_predictor'] = {
            'rf': MultiOutputRegressor(RandomForestRegressor(
                n_estimators=200, 
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42
            )),
            'gb': MultiOutputRegressor(GradientBoostingRegressor(
                n_estimators=150,
                max_depth=8,
                learning_rate=0.1,
                random_state=42
            )),
            'nn': MLPRegressor(
                hidden_layer_sizes=(128, 64, 32),
                activation='relu',
                solver='adam',
                alpha=0.001,
                learning_rate='adaptive',
                max_iter=1000,
                random_state=42
            )
        }
        
        # Modello per predizione criteri asset (9 output)
        self.models['asset_predictor'] = {
            'rf': MultiOutputRegressor(RandomForestRegressor(
                n_estimators=200,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42
            )),
            'xgb': MultiOutputRegressor(xgb.XGBRegressor(
                n_estimators=150,
                max_depth=8,
                learning_rate=0.1,
                random_state=42
            )),
            'lgb': MultiOutputRegressor(lgb.LGBMRegressor(
                n_estimators=150,
                max_depth=8,
                learning_rate=0.1,
                random_state=42,
                verbose=-1
            ))
        }
        
        # Modello per classificazione rischio finale
        self.models['risk_classifier'] = {
            'rf': RandomForestClassifier(
                n_estimators=200,
                max_depth=10,
                min_samples_split=3,
                class_weight='balanced',
                random_state=42
            ),
            'gb': GradientBoostingRegressor(
                n_estimators=150,
                max_depth=8,
                learning_rate=0.15,
                random_state=42
            )
        }
        
        # Scalers per normalizzazione
        self.scalers['threat'] = StandardScaler()
        self.scalers['asset'] = StandardScaler()
        self.scalers['risk'] = StandardScaler()
        
        print("🤖 Modelli AI configurati:")
        print(f"   - Threat Predictor: {len(self.models['threat_predictor'])} modelli")
        print(f"   - Asset Predictor: {len(self.models['asset_predictor'])} modelli") 
        print(f"   - Risk Classifier: {len(self.models['risk_classifier'])} modelli")
    
    def prepare_enhanced_features(self, threat_name, asset_name):
        """Prepara features arricchite con dati reali del framework"""
        
        # Features base
        base_features = []
        
        # Aggiungi feature semantiche
        threat_category = self.feature_engineer.get_threat_category(threat_name)
        asset_properties = self.feature_engineer.get_asset_properties(asset_name)
        affinity = self.feature_engineer.calculate_threat_asset_affinity(threat_name, asset_name)
        
        # Encoding categorico
        threat_cat_encoded = hash(threat_category) % 100  # Simple hash encoding
        segment_encoded = hash(asset_properties['segment']) % 50
        criticality_encoded = {'low': 1, 'medium': 2, 'high': 3, 'critical': 4}[asset_properties['criticality']]
        exposure_encoded = {'low': 1, 'medium': 2, 'high': 3}[asset_properties['exposure']]
        
        base_features.extend([threat_cat_encoded, segment_encoded, criticality_encoded, exposure_encoded, affinity])
        
        # Features da dati reali del framework
        enhanced_context = self.data_loader.get_enhanced_threat_context(threat_name)
        
        # CVE-based features
        cve_severity_avg = 0
        cve_count = len(enhanced_context['relevant_cves'])
        if cve_count > 0:
            cve_severity_avg = np.mean([cve['severity'] for cve in enhanced_context['relevant_cves']])
        
        # Control-based features
        control_effectiveness_avg = 0
        control_count = len(enhanced_context['applicable_controls'])
        if control_count > 0:
            control_effectiveness_avg = np.mean([ctrl['effectiveness'] for ctrl in enhanced_context['applicable_controls']])
        
        # Threat relations features
        relation_impact_avg = 0
        relation_count = len(enhanced_context['related_threats'])
        if relation_count > 0:
            relation_impact_avg = np.mean([rel['impact_modifier'] for rel in enhanced_context['related_threats']])
        
        base_features.extend([cve_severity_avg, cve_count, control_effectiveness_avg, control_count, relation_impact_avg, relation_count])
        
        return np.array(base_features, dtype=np.float32)
    
    def train_models(self, X, y_threat, y_asset, y_risk):
        """Addestra tutti i modelli AI"""
        print("🚀 Inizio addestramento modelli AI...")
        
        # Normalizzazione features
        X_scaled = self.scalers['threat'].fit_transform(X)
        
        # Split train/test
        X_train, X_test, yt_train, yt_test, ya_train, ya_test, yr_train, yr_test = train_test_split(
            X_scaled, y_threat, y_asset, y_risk, test_size=0.2, random_state=42, stratify=y_risk
        )
        
        print(f"📊 Training set: {X_train.shape[0]} samples")
        print(f"📊 Test set: {X_test.shape[0]} samples")
        
        # Addestramento modelli per predizione criteri minacce
        print("\n🎯 Training Threat Predictors...")
        threat_scores = {}
        for name, model in self.models['threat_predictor'].items():
            print(f"   Training {name}...")
            model.fit(X_train, yt_train)
            y_pred = model.predict(X_test)
            score = r2_score(yt_test, y_pred, multioutput='average')
            threat_scores[name] = score
            print(f"   {name} R² Score: {score:.4f}")
        
        # Addestramento modelli per predizione criteri asset
        print("\n🏢 Training Asset Predictors...")
        asset_scores = {}
        for name, model in self.models['asset_predictor'].items():
            print(f"   Training {name}...")
            model.fit(X_train, ya_train)
            y_pred = model.predict(X_test)
            score = r2_score(ya_test, y_pred, multioutput='average')
            asset_scores[name] = score
            print(f"   {name} R² Score: {score:.4f}")
        
        # Addestramento modelli per classificazione rischio
        print("\n⚠️ Training Risk Classifiers...")
        risk_scores = {}
        for name, model in self.models['risk_classifier'].items():
            print(f"   Training {name}...")
            if name == 'rf':  # Classificatore
                model.fit(X_train, yr_train)
                y_pred = model.predict(X_test)
                score = model.score(X_test, yr_test)
            else:  # Regressore
                model.fit(X_train, yr_train)
                y_pred = model.predict(X_test)
                # Converti predizioni continue in classi
                y_pred_class = np.round(np.clip(y_pred, 0, 4)).astype(int)
                score = np.mean(y_pred_class == yr_test)
            
            risk_scores[name] = score
            print(f"   {name} Accuracy: {score:.4f}")
        
        self.is_trained = True
        
        # Salva risultati
        self.training_results = {
            'threat_scores': threat_scores,
            'asset_scores': asset_scores,
            'risk_scores': risk_scores,
            'best_threat_model': max(threat_scores.items(), key=lambda x: x[1])[0],
            'best_asset_model': max(asset_scores.items(), key=lambda x: x[1])[0],
            'best_risk_model': max(risk_scores.items(), key=lambda x: x[1])[0]
        }
        
        print(f"\n✅ Training completato!")
        print(f"🏆 Migliori modelli:")
        print(f"   - Threat: {self.training_results['best_threat_model']} ({threat_scores[self.training_results['best_threat_model']]:.4f})")
        print(f"   - Asset: {self.training_results['best_asset_model']} ({asset_scores[self.training_results['best_asset_model']]:.4f})")
        print(f"   - Risk: {self.training_results['best_risk_model']} ({risk_scores[self.training_results['best_risk_model']]:.4f})")
        
        return self.training_results
    
    def predict_risk_assessment(self, threat_name, asset_name):
        """Predice il risk assessment completo per una coppia threat-asset"""
        if not self.is_trained:
            raise ValueError("I modelli devono essere addestrati prima della predizione!")
        
        # Prepara features
        features = self.prepare_enhanced_features(threat_name, asset_name).reshape(1, -1)
        features_scaled = self.scalers['threat'].transform(features)
        
        # Predizione criteri minacce
        best_threat_model = self.models['threat_predictor'][self.training_results['best_threat_model']]
        threat_predictions = best_threat_model.predict(features_scaled)[0]
        
        # Predizione criteri asset
        best_asset_model = self.models['asset_predictor'][self.training_results['best_asset_model']]
        asset_predictions = best_asset_model.predict(features_scaled)[0]
        
        # Predizione rischio finale
        best_risk_model = self.models['risk_classifier'][self.training_results['best_risk_model']]
        if isinstance(best_risk_model, RandomForestClassifier):
            risk_prediction = best_risk_model.predict(features_scaled)[0]
            risk_probabilities = best_risk_model.predict_proba(features_scaled)[0]
        else:
            risk_prediction_raw = best_risk_model.predict(features_scaled)[0]
            risk_prediction = int(np.round(np.clip(risk_prediction_raw, 0, 4)))
            risk_probabilities = None
        
        # Formatta risultati
        threat_results = dict(zip(THREAT_CRITERIA, np.clip(threat_predictions, 1, 5)))
        asset_results = dict(zip(ASSET_CRITERIA, np.clip(asset_predictions, 1, 5)))
        risk_level = RISK_LEVELS[risk_prediction]
        
        # Calcola confidence score
        confidence = self._calculate_confidence(features_scaled, threat_predictions, asset_predictions)
        
        return {
            'threat_scores': {k: round(v, 2) for k, v in threat_results.items()},
            'asset_scores': {k: round(v, 2) for k, v in asset_results.items()},
            'risk_level': risk_level,
            'risk_probabilities': risk_probabilities.tolist() if risk_probabilities is not None else None,
            'confidence_score': confidence,
            'enhanced_context': self.data_loader.get_enhanced_threat_context(threat_name)
        }
    
    def _calculate_confidence(self, features, threat_pred, asset_pred):
        """Calcola un confidence score per la predizione"""
        # Semplice euristica basata sulla varianza delle predizioni
        threat_var = np.var(threat_pred)
        asset_var = np.var(asset_pred)
        
        # Normalizza tra 0.5 e 1.0
        base_confidence = 0.8
        var_penalty = min(0.3, (threat_var + asset_var) / 10)
        confidence = max(0.5, base_confidence - var_penalty)
        
        return round(confidence, 3)

# Inizializza il sistema AI
ai_system = AdvancedRiskAssessmentAI(data_loader, feature_engineer)
print("🤖 Sistema AI Risk Assessment inizializzato!")

## 🚀 7. Addestramento del Modello AI

Addestriamo il sistema AI utilizzando il dataset generato basato sulla logica expert del framework CRAALSP.

In [ ]:
# Addestramento del modello AI con il dataset generato
print("🚀 Inizio processo di addestramento AI...")
print("=" * 60)

# Verifica che abbiamo il dataset
if 'X' not in locals() or X is None:
    print("❌ Dataset non trovato! Eseguire prima la generazione del dataset.")
else:
    print(f"✅ Dataset verificato: {X.shape}")
    
    # Addestra i modelli
    training_results = ai_system.train_models(X, y_threat, y_asset, y_risk)
    
    # Visualizza risultati dell'addestramento
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Performance dei modelli threat
    threat_models = list(training_results['threat_scores'].keys())
    threat_scores = list(training_results['threat_scores'].values())
    axes[0, 0].bar(threat_models, threat_scores, color='skyblue', alpha=0.7)
    axes[0, 0].set_title('Threat Predictor Performance (R² Score)')
    axes[0, 0].set_ylabel('R² Score')
    axes[0, 0].set_ylim(0, 1)
    for i, v in enumerate(threat_scores):
        axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center')
    
    # Performance dei modelli asset
    asset_models = list(training_results['asset_scores'].keys())
    asset_scores = list(training_results['asset_scores'].values())
    axes[0, 1].bar(asset_models, asset_scores, color='lightgreen', alpha=0.7)
    axes[0, 1].set_title('Asset Predictor Performance (R² Score)')
    axes[0, 1].set_ylabel('R² Score')
    axes[0, 1].set_ylim(0, 1)
    for i, v in enumerate(asset_scores):
        axes[0, 1].text(i, v + 0.01, f'{v:.3f}', ha='center')
    
    # Performance dei modelli risk
    risk_models = list(training_results['risk_scores'].keys())
    risk_scores = list(training_results['risk_scores'].values())
    axes[1, 0].bar(risk_models, risk_scores, color='orange', alpha=0.7)
    axes[1, 0].set_title('Risk Classifier Performance (Accuracy)')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].set_ylim(0, 1)
    for i, v in enumerate(risk_scores):
        axes[1, 0].text(i, v + 0.01, f'{v:.3f}', ha='center')
    
    # Learning curve del miglior modello
    best_model = ai_system.models['threat_predictor'][training_results['best_threat_model']]
    train_sizes, train_scores, val_scores = learning_curve(
        best_model, X[:1000], y_threat[:1000],  # Subset per velocità
        cv=3, train_sizes=np.linspace(0.1, 1.0, 10), scoring='r2'
    )
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)
    val_std = np.std(val_scores, axis=1)
    
    axes[1, 1].plot(train_sizes, train_mean, 'o-', color='blue', label='Training Score')
    axes[1, 1].fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    axes[1, 1].plot(train_sizes, val_mean, 'o-', color='red', label='Validation Score')
    axes[1, 1].fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
    axes[1, 1].set_title(f'Learning Curve - {training_results["best_threat_model"]}')
    axes[1, 1].set_xlabel('Training Set Size')
    axes[1, 1].set_ylabel('R² Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiche finali
    print("\\n" + "="*60)
    print("📊 RISULTATI ADDESTRAMENTO AI")
    print("="*60)
    print(f"🎯 Miglior Threat Predictor: {training_results['best_threat_model']} (R²: {training_results['threat_scores'][training_results['best_threat_model']]:.4f})")
    print(f"🏢 Miglior Asset Predictor: {training_results['best_asset_model']} (R²: {training_results['asset_scores'][training_results['best_asset_model']]:.4f})")
    print(f"⚠️ Miglior Risk Classifier: {training_results['best_risk_model']} (Acc: {training_results['risk_scores'][training_results['best_risk_model']]:.4f})")
    print(f"\\n📈 Dataset utilizzato: {X.shape[0]:,} campioni con {X.shape[1]} features")
    print(f"🎲 Combinazioni threat-asset: {len(THREATS)} × {len(ASSET_CATEGORIES)} = {len(THREATS) * len(ASSET_CATEGORIES):,}")
    
    print("\\n✅ Sistema AI pronto per le predizioni!")

## 🧪 8. Test e Validazione del Sistema AI

Testiamo il sistema AI con casi reali e validiamo le sue performance rispetto al framework CRAALSP originale.

In [ ]:
# Test del sistema AI con casi reali
def test_ai_system():
    """Testa il sistema AI con diversi scenari"""
    print("🧪 Inizio test del sistema AI...")
    
    # Casi di test rappresentativi
    test_cases = [
        ("Social Engineering", "Mission Control Commanding"),
        ("Malicious code / software / activity: Network exploit", "Space Platform Communication"),
        ("Man-in-the-Middle (MITM)", "Link Space Ground Segment"),
        ("Supply Chain Compromise", "Ground Station Reception"),
        ("Unauthorized access", "Space Platform Command Data Handling"),
        ("Denial of Service (DoS)", "Mission Control Telemetry Processing")
    ]
    
    results = []
    
    print("\\n📊 Risultati test AI:")
    print("-" * 80)
    
    for i, (threat, asset) in enumerate(test_cases, 1):
        try:
            # Predizione AI
            prediction = ai_system.predict_risk_assessment(threat, asset)
            
            # Estrai informazioni chiave
            risk_level = prediction['risk_level']
            confidence = prediction['confidence_score']
            
            # Medie dei criteri
            threat_avg = np.mean(list(prediction['threat_scores'].values()))
            asset_avg = np.mean(list(prediction['asset_scores'].values()))
            
            results.append({
                'case': i,
                'threat': threat[:40] + "..." if len(threat) > 40 else threat,
                'asset': asset[:30] + "..." if len(asset) > 30 else asset,
                'risk_level': risk_level,
                'confidence': confidence,
                'threat_avg': threat_avg,
                'asset_avg': asset_avg
            })
            
            print(f"Test {i}: {threat[:40]}{'...' if len(threat) > 40 else ''}")
            print(f"  Asset: {asset[:40]}{'...' if len(asset) > 40 else ''}")
            print(f"  🎯 Risk Level: {risk_level}")
            print(f"  📊 Confidence: {confidence:.3f}")
            print(f"  ⚡ Threat Score: {threat_avg:.2f}/5.0")
            print(f"  🏢 Asset Score: {asset_avg:.2f}/5.0")
            print()
            
        except Exception as e:
            print(f"❌ Errore nel test {i}: {str(e)}")
    
    return results

# Esegui test se il modello è addestrato
if ai_system.is_trained:
    test_results = test_ai_system()
    
    # Visualizza risultati test
    if test_results:
        df_results = pd.DataFrame(test_results)
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        
        # Distribuzione livelli di rischio
        risk_counts = df_results['risk_level'].value_counts()
        axes[0, 0].pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%')
        axes[0, 0].set_title('Distribuzione Livelli di Rischio nei Test')
        
        # Confidence scores
        axes[0, 1].bar(range(len(df_results)), df_results['confidence'], color='lightblue')
        axes[0, 1].set_title('Confidence Scores per Test Case')
        axes[0, 1].set_xlabel('Test Case')
        axes[0, 1].set_ylabel('Confidence')
        axes[0, 1].set_ylim(0, 1)
        
        # Correlazione threat vs asset scores
        axes[1, 0].scatter(df_results['threat_avg'], df_results['asset_avg'], 
                          c=range(len(df_results)), cmap='viridis', s=100)
        axes[1, 0].set_xlabel('Threat Score Average')
        axes[1, 0].set_ylabel('Asset Score Average')
        axes[1, 0].set_title('Correlazione Threat vs Asset Scores')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Performance summary
        axes[1, 1].text(0.1, 0.8, f"Test completati: {len(test_results)}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.7, f"Confidence media: {df_results['confidence'].mean():.3f}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.6, f"Threat score medio: {df_results['threat_avg'].mean():.2f}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.5, f"Asset score medio: {df_results['asset_avg'].mean():.2f}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.4, f"Rischio più comune: {df_results['risk_level'].mode()[0]}", fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Statistiche Performance')
        axes[1, 1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
    print("\\n✅ Test completati con successo!")
    
else:
    print("⚠️ Il modello deve essere addestrato prima di eseguire i test!")

## 🎮 9. Interfaccia Utente Interattiva

Creiamo un'interfaccia semplice per utilizzare il sistema AI in modo interattivo.

In [ ]:
class InteractiveRiskAssessmentInterface:
    """Interfaccia interattiva per il sistema AI di risk assessment"""
    
    def __init__(self, ai_system):
        self.ai_system = ai_system
        
    def display_welcome(self):
        """Mostra messaggio di benvenuto"""
        print("🚀" + "="*70 + "🚀")
        print("    🛰️  AI RISK ASSESSMENT SYSTEM FOR SPACE CYBERSECURITY  🛰️")
        print("    📡  Basato sul Framework CRAALSP  📡")
        print("🚀" + "="*70 + "🚀")
        print()
        print("✨ Questo sistema utilizza AI per valutare automaticamente")
        print("   i rischi cybersecurity per missioni spaziali")
        print()
    
    def list_threats(self):
        """Mostra lista delle minacce disponibili"""
        print("🎯 MINACCE DISPONIBILI:")
        print("-" * 50)
        for i, threat in enumerate(THREATS[:20], 1):  # Mostra solo le prime 20
            print(f"{i:2d}. {threat}")
        if len(THREATS) > 20:
            print(f"    ... e altre {len(THREATS) - 20} minacce")
        print()
    
    def list_assets(self):
        """Mostra lista degli asset disponibili"""
        print("🏢 ASSET DISPONIBILI:")
        print("-" * 50)
        for i, (category, sub_category, component) in enumerate(ASSET_CATEGORIES[:15], 1):  # Prime 15
            print(f"{i:2d}. {component} ({category})")
        if len(ASSET_CATEGORIES) > 15:
            print(f"    ... e altri {len(ASSET_CATEGORIES) - 15} asset")
        print()
    
    def get_user_selection(self):
        """Interfaccia per selezione utente"""
        print("🔍 SELEZIONE THREAT E ASSET")
        print("-" * 30)
        
        # Selezione threat
        print("Seleziona una minaccia (inserisci numero o nome parziale):")
        threat_input = input("Threat: ").strip()
        
        selected_threat = None
        if threat_input.isdigit():
            idx = int(threat_input) - 1
            if 0 <= idx < len(THREATS):
                selected_threat = THREATS[idx]
        else:
            # Cerca per nome parziale
            matches = [t for t in THREATS if threat_input.lower() in t.lower()]
            if matches:
                selected_threat = matches[0]
                if len(matches) > 1:
                    print(f"Trovate {len(matches)} corrispondenze, usando: {selected_threat}")
        
        if not selected_threat:
            print("⚠️ Minaccia non trovata, usando default...")
            selected_threat = THREATS[0]
        
        # Selezione asset
        print("\\nSeleziona un asset (inserisci numero o nome parziale):")
        asset_input = input("Asset: ").strip()
        
        selected_asset = None
        if asset_input.isdigit():
            idx = int(asset_input) - 1
            if 0 <= idx < len(ASSET_CATEGORIES):
                selected_asset = ASSET_CATEGORIES[idx][2]
        else:
            # Cerca per nome parziale
            matches = [(cat, sub, comp) for cat, sub, comp in ASSET_CATEGORIES 
                      if asset_input.lower() in comp.lower()]
            if matches:
                selected_asset = matches[0][2]
                if len(matches) > 1:
                    print(f"Trovate {len(matches)} corrispondenze, usando: {selected_asset}")
        
        if not selected_asset:
            print("⚠️ Asset non trovato, usando default...")
            selected_asset = ASSET_CATEGORIES[0][2]
        
        return selected_threat, selected_asset
    
    def display_assessment_result(self, threat, asset, result):
        """Mostra i risultati dell'assessment"""
        print("\\n" + "🎯" + "="*68 + "🎯")
        print("                    📊 RISULTATI RISK ASSESSMENT 📊")
        print("🎯" + "="*68 + "🎯")
        
        print(f"\\n🎯 MINACCIA: {threat}")
        print(f"🏢 ASSET: {asset}")
        print(f"⚠️ LIVELLO DI RISCHIO: {result['risk_level']}")
        print(f"📊 CONFIDENCE: {result['confidence_score']:.1%}")
        
        print("\\n" + "="*40)
        print("📋 CRITERI MINACCIA:")
        print("="*40)
        for criterion, score in result['threat_scores'].items():
            print(f"  {criterion}: {score:.2f}/5.0")
        
        print("\\n" + "="*40)
        print("📋 CRITERI ASSET:")
        print("="*40)
        for criterion, score in result['asset_scores'].items():
            print(f"  {criterion}: {score:.2f}/5.0")
        
        # Contesto arricchito
        context = result['enhanced_context']
        if context['relevant_cves']:
            print("\\n🐛 CVE RILEVANTI:")
            for cve in context['relevant_cves']:
                print(f"  - {cve['cve_id']}: {cve['description'][:60]}... (Severity: {cve['severity']})")
        
        if context['applicable_controls']:
            print("\\n🛡️ CONTROLLI APPLICABILI:")
            for ctrl in context['applicable_controls']:
                print(f"  - {ctrl['title']} (Effectiveness: {ctrl['effectiveness']:.1f}/5.0)")
        
        if context['related_threats']:
            print("\\n🔗 MINACCE CORRELATE:")
            for rel in context['related_threats']:
                print(f"  - {rel['target']} ({rel['relation']})")
        
        print("\\n" + "🎯" + "="*68 + "🎯")
    
    def run_interactive_session(self):
        """Esegue una sessione interattiva"""
        if not self.ai_system.is_trained:
            print("❌ Il sistema AI deve essere addestrato prima dell'uso!")
            return
        
        self.display_welcome()
        
        while True:
            print("\\n" + "="*50)
            print("🎮 OPZIONI:")
            print("1. 📋 Mostra minacce disponibili")
            print("2. 🏢 Mostra asset disponibili") 
            print("3. 🎯 Esegui risk assessment")
            print("4. 🧪 Esegui assessment rapido (esempi)")
            print("5. 📊 Mostra statistiche sistema")
            print("0. ❌ Esci")
            
            choice = input("\\nScegli un'opzione: ").strip()
            
            if choice == '1':
                self.list_threats()
            elif choice == '2':
                self.list_assets()
            elif choice == '3':
                threat, asset = self.get_user_selection()
                try:
                    result = self.ai_system.predict_risk_assessment(threat, asset)
                    self.display_assessment_result(threat, asset, result)
                except Exception as e:
                    print(f"❌ Errore durante l'assessment: {str(e)}")
            elif choice == '4':
                self.run_quick_examples()
            elif choice == '5':
                self.show_system_stats()
            elif choice == '0':
                print("👋 Grazie per aver utilizzato il sistema AI Risk Assessment!")
                break
            else:
                print("⚠️ Opzione non valida!")
    
    def run_quick_examples(self):
        """Esegue esempi rapidi"""
        examples = [
            ("Social Engineering", "Mission Control Commanding"),
            ("Malicious code / software / activity: Network exploit", "Space Platform Communication"),
            ("Supply Chain Compromise", "Ground Station Reception")
        ]
        
        print("\\n🧪 ESEMPI RAPIDI:")
        print("-" * 30)
        
        for i, (threat, asset) in enumerate(examples, 1):
            try:
                result = self.ai_system.predict_risk_assessment(threat, asset)
                print(f"\\n{i}. {threat[:40]}... → {asset[:30]}...")
                print(f"   Risk: {result['risk_level']} (Confidence: {result['confidence_score']:.1%})")
            except Exception as e:
                print(f"   ❌ Errore: {str(e)}")
    
    def show_system_stats(self):
        """Mostra statistiche del sistema"""
        print("\\n📊 STATISTICHE SISTEMA AI:")
        print("-" * 40)
        print(f"🎯 Minacce supportate: {len(THREATS)}")
        print(f"🏢 Asset supportati: {len(ASSET_CATEGORIES)}")
        print(f"🤖 Modelli addestrati: {len(self.ai_system.models)}")
        print(f"✅ Sistema pronto: {'Sì' if self.ai_system.is_trained else 'No'}")
        
        if hasattr(self.ai_system, 'training_results'):
            results = self.ai_system.training_results
            print(f"🏆 Miglior threat model: {results['best_threat_model']}")
            print(f"🏆 Miglior asset model: {results['best_asset_model']}")
            print(f"🏆 Miglior risk model: {results['best_risk_model']}")

# Inizializza interfaccia
interface = InteractiveRiskAssessmentInterface(ai_system)

print("🎮 Interfaccia utente pronta!")
print("   Usa interface.run_interactive_session() per iniziare")

# Esegui una demo rapida se il sistema è pronto
if ai_system.is_trained:
    print("\\n🚀 Demo rapida del sistema:")
    interface.run_quick_examples()
else:
    print("\\n⚠️ Completa prima l'addestramento del modello per utilizzare l'interfaccia!")

## 🎯 10. Utilizzo del Sistema e Conclusioni

### 🚀 Come Utilizzare il Sistema AI

1. **Esegui tutte le celle precedenti** per inizializzare e addestrare il sistema
2. **Utilizza l'interfaccia interattiva**:
   ```python
   interface.run_interactive_session()
   ```
3. **Oppure fai predizioni dirette**:
   ```python
   result = ai_system.predict_risk_assessment("Social Engineering", "Mission Control Commanding")
   print(result)
   ```

### 📊 Caratteristiche del Sistema

- **🎯 Dataset**: Basato sulla logica expert del framework CRAALSP
- **🤖 Modelli**: Ensemble di Random Forest, XGBoost, LightGBM e Neural Networks
- **📡 Dati Reali**: Integrazione con CSV del framework (Threat.csv, Control.csv, etc.)
- **🔄 Validazione**: Cross-validation e test su casi reali
- **⚡ Performance**: Predizioni in tempo reale con confidence scores

### 🎓 Applicazioni Pratiche

1. **Risk Assessment Automatico**: Valutazione rapida di nuove minacce
2. **Decision Support**: Supporto per analisti di cybersecurity
3. **Prioritizzazione**: Identificazione di combinazioni threat-asset ad alto rischio
4. **Scenario Analysis**: Valutazione di impatti di attacchi complessi
5. **Training**: Addestramento di personale su scenari realistici

### 🔬 Vantaggi vs Framework Tradizionale

- **⚡ Velocità**: Predizioni istantanee vs valutazione manuale
- **🎯 Consistenza**: Eliminazione della variabilità umana
- **📈 Scalabilità**: Gestione di migliaia di combinazioni
- **🔄 Adattabilità**: Aggiornamento continuo con nuovi dati
- **📊 Quantificazione**: Scores numerici e confidence metrics

### 🛠️ Estensioni Future

- **🌐 API REST**: Integrazione con sistemi esterni
- **📱 Web Interface**: Dashboard interattivo
- **🔄 Continuous Learning**: Aggiornamento automatico con feedback
- **🎯 Specialized Models**: Modelli specifici per tipi di missione
- **🤝 Integration**: Connessione diretta con il framework CRAALSP

---

**✅ Il sistema AI per Risk Assessment Spaziale è ora pronto per l'uso!**

In [ ]:
# 🎯 DEMO FINALE E ISTRUZIONI PER L'USO
print("🚀" + "="*80 + "🚀")
print("    🎯 AI RISK ASSESSMENT SYSTEM FOR SPACE CYBERSECURITY - READY! 🎯")
print("🚀" + "="*80 + "🚀")

print("""
✨ SISTEMA COMPLETAMENTE OPERATIVO! ✨

🎮 Per utilizzare l'interfaccia interattiva:
   >>> interface.run_interactive_session()

🤖 Per predizioni dirette:
   >>> result = ai_system.predict_risk_assessment("threat_name", "asset_name")
   >>> print(result)

🧪 Per eseguire test rapidi:
   >>> interface.run_quick_examples()

📊 Per vedere le statistiche:
   >>> interface.show_system_stats()

📋 Esempi di minacce:
   - "Social Engineering"
   - "Malicious code / software / activity: Network exploit"
   - "Man-in-the-Middle (MITM)"
   - "Supply Chain Compromise"

🏢 Esempi di asset:
   - "Mission Control Commanding"
   - "Space Platform Communication"
   - "Ground Station Reception"
   - "Link Space Ground Segment"

🎯 FUNZIONALITÀ PRINCIPALI:
   ✅ Predizione automatica dei 7 criteri di minaccia
   ✅ Predizione automatica dei 9 criteri di asset
   ✅ Classificazione del livello di rischio finale
   ✅ Integrazione con dati reali del framework CRAALSP
   ✅ Confidence scores per validare le predizioni
   ✅ Contesto arricchito con CVE e controlli applicabili

🚀 Il sistema è pronto per essere utilizzato nella tua ricerca!
""")

# Esegui una demo finale se tutto è pronto
if ai_system.is_trained:
    print("🎬 DEMO FINALE - Risk Assessment Automatico:")
    print("-" * 50)
    
    demo_cases = [
        ("Social Engineering", "Mission Control Commanding"),
        ("Supply Chain Compromise", "Space Platform Communication")
    ]
    
    for threat, asset in demo_cases:
        try:
            result = ai_system.predict_risk_assessment(threat, asset)
            print(f"\\n🎯 {threat} → {asset}")
            print(f"   📊 Risk Level: {result['risk_level']}")
            print(f"   🎲 Confidence: {result['confidence_score']:.1%}")
            print(f"   ⚡ Avg Threat Score: {np.mean(list(result['threat_scores'].values())):.2f}/5.0")
            print(f"   🏢 Avg Asset Score: {np.mean(list(result['asset_scores'].values())):.2f}/5.0")
        except Exception as e:
            print(f"   ❌ Errore: {str(e)}")
    
    print("\\n🎉 Sistema AI Risk Assessment completamente funzionante!")
    print("🎓 Pronto per essere utilizzato nella tua tesi!")
else:
    print("⚠️ Ricorda di eseguire l'addestramento prima di utilizzare il sistema!")

print("\\n" + "🚀" + "="*80 + "🚀")